# Linear Regression Custom Code Example

In [9]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression

In [7]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [10]:
spark

In [13]:
data = spark.read.csv('./data/Ecommerce_Customers.csv',inferSchema = True, header = True)

In [15]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [18]:
for item in data.head(1)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


## Setting Up DataFrame for Machine Learning

In [19]:
# A few things we need to do before Spark can accept the data!
# It needs to be in the form of two columns
# ("label","features")

# Import VectorAssembler and Vectors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [20]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [22]:
assembler = VectorAssembler(inputCols = ['Avg Session Length','Time on App','Time on Website',
                                          'Length of Membership'],outputCol= 'features')

In [23]:
output = assembler.transform(data)

In [25]:
output.head()

Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))

In [26]:
final_data = output.select('features','Yearly Amount Spent')

In [27]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [28]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [29]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                365|
|   mean|   499.224451811217|
| stddev|  80.92450474341202|
|    min|   266.086340948469|
|    max|  744.2218671047146|
+-------+-------------------+



In [31]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                135|
|   mean|  499.5562534681574|
| stddev|  75.08012295465312|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [34]:
lr = LinearRegression(labelCol = 'Yearly Amount Spent')

In [35]:
lr_model = lr.fit(train_data)

In [36]:
test_results = lr_model.evaluate(test_data)

In [37]:
#Difference between the predicted value and the actual label from the test_data.
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| 10.104412520986443|
|-3.9633767118586434|
|-21.482702771722074|
| 18.235885761169186|
| 3.6472162443516254|
|0.21055841952920673|
| -8.268715642350458|
|   -5.7267730802713|
| -4.041106092661778|
|-0.7799654811047958|
| -8.157202540250069|
| -6.107007251806351|
|-11.661247404671258|
|-17.656972319494855|
|-1.9811088759401514|
|  1.241116018237335|
|  8.541385674374226|
|-2.5720811380924715|
| -6.746793838014241|
|  17.40926163371296|
+-------------------+
only showing top 20 rows



In [39]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))
print("R2:{}".format(test_results.r2))

RMSE: 8.823750836209168
MSE: 77.85857881950199
R2:0.986084926907766


In [40]:
unlabeled_data = test_data.select('features')

In [41]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[29.5324289670579...|
|[30.8794843441274...|
|[31.1239743499119...|
|[31.3123495994443...|
|[31.3662121671876...|
|[31.3895854806643...|
|[31.4474464941278...|
|[31.5257524169682...|
|[31.6253601348306...|
|[31.6610498227460...|
|[31.7207699002873...|
|[31.7242025238451...|
|[31.8093003166791...|
|[31.8164283341993...|
|[31.8186165667690...|
|[31.8293464559211...|
|[31.8512531286083...|
|[31.8530748017465...|
|[31.9453957483445...|
|[32.0180740106320...|
+--------------------+
only showing top 20 rows



In [42]:
predictions = lr_model.transform(unlabeled_data)

In [43]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[29.5324289670579...|398.53593855164104|
|[30.8794843441274...| 494.1699766967133|
|[31.1239743499119...|508.42975661148785|
|[31.3123495994443...|445.35553226677143|
|[31.3662121671876...| 426.9416663121333|
|[31.3895854806643...| 409.8590526404537|
|[31.4474464941278...| 426.8714577375745|
|[31.5257524169682...| 449.6923998901532|
|[31.6253601348306...|  380.378006849586|
|[31.6610498227460...|417.13831906100563|
|[31.7207699002873...|  546.932136018273|
|[31.7242025238451...|509.49489453976685|
|[31.8093003166791...| 548.4331467675124|
|[31.8164283341993...| 518.7794638231512|
|[31.8186165667690...| 448.3997822460758|
|[31.8293464559211...|383.91122196973765|
|[31.8512531286083...|464.45086099242417|
|[31.8530748017465...|461.85720460044445|
|[31.9453957483445...| 663.7667177756662|
|[32.0180740106320...|340.37384911160234|
+--------------------+------------

In [ ]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))
print("R2:{}".format(test_results.r2))